In [20]:
import arcrest
import uuid
import json
from IPython.display import IFrame

def show_web_map(id):
    url = 'http://www.arcgis.com/home/webmap/viewer.html?webmap=' + id
    return IFrame(url, width='100%', height=500)

# Demo Values
randomName = "map_{0}".format(uuid.uuid4().hex)
featureSet = {"geometryType":"esriGeometryPoint","features":[]}
fields = [{"name":"City","alias":"CityName","type":"esriFieldTypeString",
           "length":255,"editable":True,"Noneable":True,"domain":None}]
d = {"operationalLayers":[{"id":"csv_1429","title":"Esri Redlands","featureCollection":{"layers":[
    {"layerDefinition":{"geometryType":"esriGeometryPoint",
                        "objectIdField":"__OBJECTID","type":"Feature Layer",
                        "typeIdField":"",
                        "drawingInfo":{"renderer":{"type":"simple",
                                                   "symbol":{
                                                       "type": "esriSMS",
                                                           "style": "esriSMSCircle",
                                                           "color": [255,0,0,255],
                                                           "size": 5,
                                                           "angle": 0,
                                                           "xoffset": 0,
                                                           "yoffset": 0,
                                                           "outline":
                                                               {
                                                                   "color": [0,0,0,255],
                                                                   "width": 1    }
                                                       }},"fixedSymbols":True},
                        "fields":[{"name":"__OBJECTID","alias":"__OBJECTID","type":"esriFieldTypeOID",
                                   "editable":False,"Noneable":False,"domain":None}],
                        "types":[],"capabilities":"Query","name":"Redlands",
                        "templates":[]},"featureSet":{"features":[],"geometryType":"esriGeometryPoint"},
     "nextObjectId":4,"popupInfo":{}}],"showLegend":True},"visibility":True,"opacity":1}],
     "baseMap":{"baseMapLayers":[
         {"id":"NatGeo_World_Map_1836","opacity":1,"visibility":True,
          "url":"http://services.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer"}],
                "title":"National Geographic"},
     "version":"1.6"
     }

In [21]:
username = ""
password = ""
sh = arcrest.AGOLTokenSecurityHandler(username=username, password=password)
admin = arcrest.manageorg.Administration(securityHandler=sh)


In [22]:
#What version of AGOL are we using?
print(admin.currentVersion)

4.1


In [23]:
#Let's view the portal properties
portal = admin.portals.portal()

In [24]:
geocode_url = portal.helperServices['geocode'][0]['url']
print (geocode_url)

https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer


In [25]:
geocode = arcrest.ags.GeocodeService(url=geocode_url, securityHandler=sh)

In [26]:
locations = geocode.find(text="380 New York St, Redlands, CA 92373")

In [27]:
for l in locations['locations']:
    featureSet['features'].append(l['feature'])
    del l
# Add the location to the operationLayer
d['operationalLayers'][0]['featureCollection']['layers'][0]['featureSet'] = featureSet
d['operationalLayers'][0]['featureCollection']['layers'][0]['layerDefinition']['fields'] = fields


In [29]:
# Add the web map to AGOL
#
admin = arcrest.manageorg.Administration(securityHandler=sh)
# Access my user content
myself = admin.content.users.user()
# Set the parameters for the item
ip = arcrest.manageorg.ItemParameter()
ip.title = randomName
ip.type = "Web Map"
ip.extent = [[-117.3009,34.0102],[-117.0474,34.1001]]
# add the item
item = myself.addItem(itemParameters=ip, text=json.dumps(d))
# share it out
item.shareItem(everyone=True)
mymap = item.id
#show the map in IPython
show_web_map(mymap)
